In [2]:
import os
import openai

## Parse HTML

In [11]:
from langchain.document_loaders import BSHTMLLoader

documents = []

for file in os.listdir("../html"):
    loader = BSHTMLLoader(os.path.join("../html", file))
    data = loader.load()
    documents.append(data[0])


## Split Documents
Split the document into sentences.

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 450,
    chunk_overlap = 15,
    separators=["\n\n", "\n", " ", ""]
)

sentences = text_splitter.split_documents(documents)
